In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

# .env 파일에서 환경변수 불러옴
load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 KBO 야구 전문가 AI야. 사용자가 원하는 야구팀을 순위로 나타내줘?"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

messages = chat_template.format_messages(text="2025년 포스트시즌에 들어갈 야구 5팀을 말해줘")
answer = llm.invoke(messages)
print(answer.content)


2025년 KBO 포스트시즌에 진출할 팀을 예측하는 것은 여러 변수에 따라 달라질 수 있지만, 최근 몇 년간의 성적과 팀 구성, 선수 영입 등을 고려하여 예측해볼 수 있습니다. 다음은 2025년 포스트시즌에 진출할 가능성이 높은 5팀입니다:

1. **SSG 랜더스** - 최근 몇 년간 강력한 전력을 유지하고 있어 포스트시즌 진출 가능성이 높습니다.
2. **LG 트윈스** - 꾸준한 성적을 내고 있으며, 젊은 선수들의 성장도 기대됩니다.
3. **키움 히어로즈** - 강력한 타선과 안정적인 투수진을 보유하고 있어 경쟁력이 있습니다.
4. **KT 위즈** - 최근 몇 년간의 성적이 좋고, 팀의 전력이 계속해서 강화되고 있습니다.
5. **삼성 라이온즈** - 전통적인 강팀으로, 선수단의 재편성과 함께 포스트시즌 진출을 노릴 수 있습니다.

이 예측은 현재의 팀 상황과 선수 구성에 기반한 것이며, 실제 결과는 시즌 진행에 따라 달라질 수 있습니다.


In [8]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

load_dotenv()

# csv 파서 선언
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List {number} {subject}. answer in Korea \n{format_instructions}",
    input_variables=["subject", "number"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser
chain.invoke({"subject": "공포 영화", "number": "4"})

['곡성', '사바하', '장화', '빨간 신']

## JSON Parser

In [23]:
from typing import List
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
import os

# 1. .env 파일에서 OPENAI_API_KEY 로드
load_dotenv()

# 2. 데이터 구조 정의 (Pydantic)
class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")

# 3. 쿼리 정의
country_query = "대한민국은 어떤 나라야?"

# 4. JSON 파서 설정
parser = JsonOutputParser(pydantic_object=Country)

# 5. 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 6. 모델 초기화 (API 키는 .env에서 자동으로 인식됨)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 7. 체인 구성 및 실행
chain = prompt | model | parser
result = chain.invoke({"query": country_query})

# 8. 결과 출력
print(result)


{'continent': 'Asia', 'population': '51780579'}
